In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [13]:
# Creating the dataset
def generate_dummy_data(num_asteroids, num_comets, ast_min, ast_max):
    # Generate dummy data for asteroids
    asteroid_names = ['Asteroid{}'.format(i) for i in range(1, num_asteroids + 1)]
    asteroids = pd.DataFrame({
        'Name': np.random.choice(asteroid_names, size=num_asteroids, replace=False),
        'Diameter (km)': np.random.uniform(ast_dia_min, ast_dia_max, size=num_asteroids),
        'Period of Revolution (years)': np.random.uniform(ast_min, ast_max, size=num_asteroids),
        'Mass (MTonne)': np.random.uniform(ast_mass_min, ast_mass_max, size=num_asteroids),
        'Max Distance from Sun (AU)': np.random.uniform(max_range_min, max_range_max, size=num_asteroids),
        'Min Distance from Sun (AU)': np.random.uniform(min_range_min, min_range_max, size=num_asteroids),
        'Type': ['Asteroid'] * num_asteroids
    })

    # Generate dummy data for comets
    comet_names = ['Comet{}'.format(i) for i in range(1, num_comets + 1)]
    comets = pd.DataFrame({
        'Name': np.random.choice(comet_names, size=num_comets, replace=False),
        'Diameter (km)': np.random.uniform(com_dia_min, com_dia_max, size=num_comets),
        'Period of Revolution (years)': np.random.uniform(com_range_min, com_range_max, size=num_comets),
        'Mass (MTonne)': np.random.uniform(com_mass_min, com_mass_max, size=num_comets),
        'Max Distance from Sun (AU)': np.random.uniform(max_range_min, max_range_max, size=num_comets),
        'Min Distance from Sun (AU)': np.random.uniform(min_range_min, min_range_max, size=num_comets),
        'Type': ['Comet'] * num_comets
    })

    planets = pd.DataFrame({
    'Name': ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune'],
    'Diameter (km)': [4879, 12104, 12756, 6792, 139820, 116460, 50724, 49244],
    'Period of Revolution (years)': [0.24, 0.62, 1, 1.88, 11.86, 29.46, 84.01, 164.8],
    'Mass (MTonne)': [3.30e14, 4.87e15, 5.97e15, 6.42e14, 1.90e18, 5.68e17, 8.68e16, 1.02e17],
    'Max Distance from Sun (AU)': [0.47, 0.73, 1.02, 1.67, 5.46, 10.12, 20.08, 30.07],
    'Min Distance from Sun (AU)': [0.31, 0.71, 0.98, 1.38, 4.95, 9.01, 18.33, 29.81],
    'Type': ['Planet', 'Planet', 'Planet', 'Planet', 'Planet', 'Planet', 'Planet', 'Planet']})

    # Concatenate the dataframes into a single dataset
    df = pd.concat([asteroids, comets, planets])

    # Reset the index of the final dataset
    df.reset_index(drop=True, inplace=True)

    return df

# generate object names
import random

names_asteroid = ['Apollos', 'Amors', 'Atens', 'Hungarias', 'Mars Crossers', 'Ceres','Vesta','Pallas','Eros','Apophis']
names_comet = ['Halley', 'Hale-Bopp', 'Encke', 'McNaught', 'Ikeya-Zhang','Hyakutake','Borrelly','Chiron','Holmes','Bennett']

def generate_object_names(num_names, object_list):
    names = []
    for _ in range(num_names):
        name = ''
        # Generate a random number of name components (2-4) for each name
        num_components = random.randint(2, 4)

        for _ in range(num_components):
            # Generate a random name component (e.g., first name, middle name, last name)
            name_component = random.choice(object_list)
            name += name_component + ' '

        names.append(name.strip())

    return names

# Example usage
num_asteroids, num_comets = 500, 450

com_range_min, com_range_max = 3.3, 330
com_dia_min, com_dia_max = 5, 70
com_mass_min, com_mass_max = 1e04, 5e05
ast_mass_min, ast_mass_max = 1e02, 7e08
max_range_min, max_range_max = 400, 960
min_range_min, min_range_max = 0.35 , 100
ast_min, ast_max = 3, 6
ast_dia_min, ast_dia_max = 0.1, 530

asteroid_names = generate_object_names(num_asteroids , names_asteroid)
comet_names = generate_object_names(num_comets , names_comet)

df = generate_dummy_data(num_asteroids, num_comets, ast_min, ast_max)

# Replace the generic names with actual names
df.loc[df['Type'] == 'Asteroid', 'Name'] = np.random.choice(asteroid_names, size=num_asteroids, replace=False)
df.loc[df['Type'] == 'Comet', 'Name'] = np.random.choice(comet_names, size=num_comets, replace=False)


In [14]:
# Map labels to numeric values
type_mapping = {'Asteroid': 0, 'Comet': 1, 'Planet': 2}
df['Type_label'] = df['Type'].map(type_mapping)

# Split the dataset into features and labels
features = df.drop(['Name', 'Type', 'Type_label'], axis=1).values
labels = df['Type_label'].values

# Convert the labels to categorical
num_classes = len(set(labels))
labels = tf.keras.utils.to_categorical(labels, num_classes)

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

In [15]:
train_features.shape, test_features.shape, train_labels.shape, test_labels.shape

((766, 5), (192, 5), (766, 3), (192, 3))

In [16]:
# Create a sequential model
model = Sequential()
model.add(Dense(81, activation='relu', input_shape=(train_features.shape[1],)))
model.add(Dense(9, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_features, train_labels, epochs=10, batch_size=1)

Epoch 1/10
766/766 [==============================] - 2s 2ms/step - loss: 0.3880 - accuracy: 0.8982
Epoch 2/10
766/766 [==============================] - 2s 2ms/step - loss: 0.1250 - accuracy: 0.9648
Epoch 3/10
766/766 [==============================] - 2s 2ms/step - loss: 0.0850 - accuracy: 0.9765
Epoch 4/10
766/766 [==============================] - 1s 2ms/step - loss: 0.0701 - accuracy: 0.9804
Epoch 5/10
766/766 [==============================] - 1s 2ms/step - loss: 0.0625 - accuracy: 0.9830
Epoch 6/10
766/766 [==============================] - 1s 2ms/step - loss: 0.0555 - accuracy: 0.9830
Epoch 7/10
766/766 [==============================] - 1s 2ms/step - loss: 0.0523 - accuracy: 0.9856
Epoch 8/10
766/766 [==============================] - 1s 2ms/step - loss: 0.0457 - accuracy: 0.9883
Epoch 9/10
766/766 [==============================] - 1s 2ms/step - loss: 0.0403 - accuracy: 0.9883
Epoch 10/10
766/766 [==============================] - 1s 2ms/step - loss: 0.0437 - accuracy: 0.9856

In [27]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_features, test_labels)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

6/6 [==============================] - 0s 3ms/step - loss: 0.0599 - accuracy: 0.9844
Test loss: 0.05985721945762634
Test accuracy: 0.984375


In [30]:
# taking random label from the dataset

n = np.random.randint(low=0, high=52)

# respective feature value array converted into list
x_list = list(test_features[n])
print(x_list)
# prediction is performed with the model and shape of galaxy is displayed
predict = model.predict([x_list])
predict
val_list = []
for n in range(3):
    val_list.append(predict[0][n])

target = np.max(val_list)
tolerance = 1e-6

index = None
for i, val in enumerate(val_list):
    if abs(val - target) < tolerance:
        index = i
        break


if index ==0:
    print('Asteroid')
elif index==1:
    print('Comet')
else:
    print("Planet")


[-0.03871520410693967, -0.7174384141494999, -0.03814274769375168, 1.0204558062665319, -1.2647759202349316]
1/1 [==============================] - 0s 50ms/step
Asteroid


### Conclusion: a deep learning predictive model is developed with more than 98% accuracy which can predict the type of celestial object based on its properties.
#### A better model can be developed if we get real data instead of dummy data.